In [1]:
import numpy as np
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, LeavePOut, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder, minmax_scale, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_text
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Activation, Flatten
from sklearn.metrics import (accuracy_score,confusion_matrix, ConfusionMatrixDisplay, r2_score, silhouette_score, 
balanced_accuracy_score,cohen_kappa_score, recall_score, precision_score, f1_score)
from kneed import knee_locator, KneeLocator
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("E:/Python Programs/EuroAstro/new_y1.csv")
df.head()

,Date_,Month,Date,Year,Weekday,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,1st Num
0,1/1/2021,1,1,2021,Friday,20,40,9,2,80,84,72,84,5,54,sixteentwenties
1,1/5/2021,1,5,2021,Tuesday,30,60,26,2,42,84,19,84,5,54,elevenfifteens
2,1/8/2021,1,8,2021,Friday,40,80,48,3,63,22,38,84,5,54,sixteentwenties
3,1/12/2021,1,12,2021,Tuesday,11,21,80,3,22,22,76,84,5,54,onefives
4,1/15/2021,1,15,2021,Friday,11,42,92,4,66,22,20,84,5,54,onefives


In [3]:
df.drop(labels=['Date_'],axis=1,inplace=True)

In [4]:
encoding = LabelEncoder()
df["Weekday"] = encoding.fit_transform(df["Weekday"])
df["1st Num"] = encoding.fit_transform(df["1st Num"])

In [5]:
df["1st Num"].value_counts()

1st Num
1    140
3    100
0     59
2     42
5     25
6      5
4      1
Name: count, dtype: int64

In [6]:
# 0 means Friday; 1 means Tuesday
# 0 means 2022; 1 means 2023; 2 means 2024
# 0 means elevenfifteens, 1 means onefives, 2 means sixteentwenties, 3 means sixtens, 4 means thirtyonefives,
# 5 means twentyonefives, 6 means twentysixthirties

df.head()

,Month,Date,Year,Weekday,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,1st Num
0,1,1,2021,0,20,40,9,2,80,84,72,84,5,54,2
1,1,5,2021,1,30,60,26,2,42,84,19,84,5,54,0
2,1,8,2021,0,40,80,48,3,63,22,38,84,5,54,2
3,1,12,2021,1,11,21,80,3,22,22,76,84,5,54,1
4,1,15,2021,0,11,42,92,4,66,22,20,84,5,54,1


In [7]:
X = df.iloc[:,0:14]
y = df.iloc[:,14:]

In [8]:
X.head()

,Month,Date,Year,Weekday,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu
0,1,1,2021,0,20,40,9,2,80,84,72,84,5,54
1,1,5,2021,1,30,60,26,2,42,84,19,84,5,54
2,1,8,2021,0,40,80,48,3,63,22,38,84,5,54
3,1,12,2021,1,11,21,80,3,22,22,76,84,5,54
4,1,15,2021,0,11,42,92,4,66,22,20,84,5,54


In [9]:
y.head()

,1st Num
0,2
1,0
2,2
3,1
4,1


In [10]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=['Month','Date','Year','Weekday','Lagna','Sun','Moon','Mars','Mercury','Jupiter','Venus','Saturn','Rahu','Ketu'])
X_scaled.head()

,Month,Date,Year,Weekday,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu
0,-1.505293,-1.668858,-1.254874,-1.0,-0.615965,0.183601,-0.964322,-1.059961,1.725913,1.236511,1.475956,1.588035,-0.486488,0.936317
1,-1.505293,-1.213381,-1.254874,1.0,-0.159967,0.910964,-0.305038,-1.059961,0.318906,1.236511,-0.586115,1.588035,-0.486488,0.936317
2,-1.505293,-0.871773,-1.254874,-1.0,0.296031,1.638327,0.548152,-1.021281,1.096462,-0.543361,0.153118,1.588035,-0.486488,0.936317
3,-1.505293,-0.416296,-1.254874,1.0,-1.026363,-0.507394,1.789156,-1.021281,-0.421624,-0.543361,1.631584,1.588035,-0.486488,0.936317
4,-1.505293,-0.074688,-1.254874,-1.0,-1.026363,0.256337,2.254532,-0.982601,1.207542,-0.543361,-0.547208,1.588035,-0.486488,0.936317


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, random_state=10)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((279, 14), (279, 1), (93, 14), (93, 1))

In [12]:
pca = PCA(n_components=2)

In [13]:
X_train_trf = pca.fit_transform(X_train)
X_test_trf = pca.transform(X_test)

In [14]:
X_train_trf.shape

(279, 2)

In [15]:
X_train_trf

array([[ 4.22069332e+00,  1.67829910e+00],
       [-2.35302923e-01,  1.54972989e+00],
       [ 5.54745028e-01,  2.02423363e+00],
       [-2.03415518e+00,  1.31289577e+00],
       [ 5.37635866e-01,  8.78234337e-01],
       [-2.01135865e+00, -5.46510682e-01],
       [-1.01721554e-01,  6.93156410e-01],
       [-2.66350212e-01, -4.61687376e-01],
       [-4.56015439e-01,  1.91691633e+00],
       [ 2.41831993e+00,  3.97915033e-01],
       [-3.10976339e-01, -9.29310499e-01],
       [-1.94661105e+00, -1.00414129e+00],
       [ 7.48675229e-02, -2.37955604e+00],
       [-7.18224770e-01, -6.60040682e-01],
       [ 1.97035555e+00, -9.16631282e-01],
       [-9.55782849e-01, -1.22407982e-01],
       [-9.57417761e-01, -1.50571436e+00],
       [-2.51653997e-01, -1.83564103e+00],
       [-2.21471212e-01,  1.01137098e+00],
       [-1.10726343e+00, -3.62511464e-01],
       [-2.01215059e+00, -3.30191485e-01],
       [-3.64712094e-01,  1.87814033e+00],
       [-2.40712429e+00, -4.62318039e-02],
       [-9.

In [47]:
model = Sequential()

model.add(Dense(100, activation='relu', input_dim = 2))
model.add(Dropout(0.1))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                     │ (None, 100)                 │             300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 200)                 │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 200)                 │          40,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 60,901 (237.89 KB)

 Trainable params: 60,901 (237.89 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [49]:
model.fit(X_train_trf, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.3618 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3269 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3191 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3436 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3285 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3337 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3258 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3316 - l

In [50]:
model.layers[0].get_weights()

[array([[-0.1574979 , -0.0148825 , -0.0673081 ,  0.16728899, -0.24304028,
          0.14293046, -0.23922716,  0.14294697,  0.16226122, -0.24572748,
         -0.24762338, -0.24241917,  0.25773844,  0.10884549, -0.29051667,
         -0.30624574,  0.24708597, -0.27514225, -0.16046903,  0.24552897,
         -0.28217864,  0.18452953, -0.2117915 ,  0.15565202,  0.23066612,
         -0.16393366,  0.25010243,  0.2851903 ,  0.34304512, -0.09133989,
          0.18131809,  0.22311439, -0.15367724,  0.12671404, -0.33437297,
          0.27709192,  0.22469375, -0.33337638,  0.31125748, -0.26906136,
          0.20803335, -0.09746805, -0.24722594,  0.26956758,  0.18214703,
          0.28560826, -0.3298169 ,  0.12276646,  0.2694843 ,  0.09946907,
         -0.20382014,  0.30701077,  0.06026047, -0.2771535 ,  0.30939752,
         -0.33770844, -0.13302384,  0.23468779,  0.01364073,  0.29998025,
          0.23386677, -0.23133121,  0.21121295,  0.23804526,  0.25625947,
          0.26043668,  0.12351186,  0.

In [51]:
y_predict = model.predict(X_test_trf, batch_size=64)
y_predict

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [52]:
model.evaluate(X_test_trf,y_test,batch_size=64)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4313 - loss: 0.0000e+00 


[0.0, 0.4516128897666931]

In [66]:
model = Sequential()

model.add(Dense(100, activation='relu', input_dim = 14))
model.add(Dropout(0.1))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_44 (Dense)                     │ (None, 100)                 │           1,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 200)                 │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 200)                 │          40,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 62,101 (242.58 KB)

 Trainable params: 62,101 (242.58 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [68]:
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.2157 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3529 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3217 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3326 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3503 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3482 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3488 - loss: 0.0000e+00 - val_accuracy: 0.3571 - val_loss: 0.0000e+00
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3352 - lo

In [70]:
y_predict = model.predict(X_test, batch_size=64)
y_predict

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [71]:
model.evaluate(X_test,y_test,batch_size=64)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4313 - loss: 0.0000e+00 


[0.0, 0.4516128897666931]

In [73]:
# Pearson's correlation

df.corr()['1st Num']

Month      0.003952
Date       0.057686
Year       0.021665
Weekday   -0.042836
Lagna     -0.007729
Sun        0.014634
Moon      -0.051946
Mars       0.083608
Mercury    0.100102
Jupiter    0.062639
Venus      0.089558
Saturn     0.025476
Rahu      -0.016379
Ketu      -0.008261
1st Num    1.000000
Name: 1st Num, dtype: float64